# Code Structure and guide:

#### Confused?? Ctrl+Click on a function/class to see arguments and docstrings. Don't forget to scroll up to the Base Class, if you're using a child class. 

#### See power_lengthV5.ipynb for a complex example (3D phase diagram)

## What is a measurement? 
* A measurement occurs when the Measure class is initialised, and the take_measurement function is run. This exists inside measure.py.
* A measurement consists of one set of data taken from multiple components at a point in time, e.g. one image from a camera and one spectrometer reading. 
* A measurement will be associated with a unique timestamp. 
* Each measurement will have a 'meta' file, which is a dictionary containing important parameters for each measurement, e.g. current laser power, camera exposure time etc. 
* This dictionary lives instead components.py, and is called params. To add values to this dictionary:

In [ ]:
#If inside components.py
value=1
params.update({'new_parameter': value})

In [4]:
#If inside a different file e.g. Measure.py or the running loop file
import Components as comp
value=1
comp.params.update({'new_parameter': value}) #The meta parameter dictionary is always accessible

* With each call of take_measurement(), the most recent meta parameter dictionary is saved, and identified with the unique timestamp

## How to run an experiment?
1) Import
2) Initialise non-sensitive components
3) Run take_measurement() or a more complex running loop

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import logging
import winsound
import numpy as np
from tqdm.notebook import tqdm
import Components as comp
from Measure import Measure, power_scan #Import the Measure object from Measure.py
import socket
import sys
if socket.gethostname() == "ph-photonbec5": #You may need to append paths to make this work, e.g. depending on where pbec_analysis is
    sys.path.append(r"D:/Control/PythonPackages/")

from pbec_analysis import make_timestamp

### Initialise components

* Create a python dictionary of the components. The names of each component in the dictionary should correspond to names of functions in Measure.py (if those devices should take readings of data), or correspond to naming convention used in measure.py. 

* Only initialise components that aren't sensitive to opening and closing (don't complain that 'device has already been opened' when connecting multiple times) - those devices should be opened in a 'with' statement inside the main loop

In [ ]:
components = dict()

components.update({"laser": comp.Toptica_Laser(com_port='COM12')})
components.update({"powermeter": comp.Thor_PowerMeter(power_meter_usb_name='USB0::0x1313::0x8078::P0034379::INSTR', num_power_readings=100, bs_factor=0.5, wavelength=950)})

#Increase initial time for lower intensity signals, increase total time for more averages. Note, uncertainties are likely very small - do not bother calculating. 
components.update({"spectrometer": comp.Spectrometer(spec_nd=1 / 30.5, total_time=100, initial_time=1000, min_lamb=910)}) 
components.update({"wheel": comp.FilterWheel(com_port='COM9', allowed_filter_positions = [0, 5])}) #Can increase the number of filter positions

### Take Measurement 
* Initialise components that are sensitive to being opened multiple times using 'with', initialise the measure object and take a measurement
* Current components that require 'with' are cameras (errors with continuous acquisition) and the TranslationStage

In [ ]:
time_stamps = []

#Instead of printing troubleshooting info e.g. pca value, this information is saved in a logging file
logging.basicConfig(filename = fr'Logs\Example_{make_timestamp}.log', filemode='w', level=logging.DEBUG, force=True) 
print('hi')

#This bit takes and saves the measurement!
with comp.FLIR_Camera(measure=True, algorithm='rising') as camera: #Open sensitive objects using 'with'
    measure = Measure(comps=components) #Send avaliable components to Measure object
    ts = measure.take_measurement() #Actually take the measurement



print(ts) #Print the timestamp of the measurement, identifies the file name of the meta parameter, image and spectrometer
winsound.Beep(1000, 2000)


You can easily take a series of measurements e.g. 

Example power scan:

In [ ]:
p_list = np.linspace(5e-3, 245e-3, 20)

with comp.FLIR_Camera(measure=True, algorithm='rising') as camera: 
    time_stamps = []
    components['wheel'].reset() #Reset the filter wheel outside the measurement
    for pwr in tqdm(p_list, leave=True):
        # Reset
        components['laser'].set(pwr)
        logging.info('PCA:', pca)
        # Set up measure class
        measure = Measure(components, PCA=pca)
        # Take measurement
        timestamp = measure.take_measurement()
        time_stamps.append(timestamp)

    logging.info(time_stamps[0], time_stamps[-1]) #Save the first and last timestamps, which identify the measurement!


In [ ]:
p_list = np.linspace(5e-3, 245e-3, 20)


#Alternatively use the convience function power_scan instead measure.py:
with comp.FLIR_Camera(measure=True, algorithm='rising') as camera: 
    power_scan(p_list, components)

Create custom measurement loops to alter any variable you choose!

# How does it actually work?
1) Create the Measure object, using the list of components defined for the measurement. This object will create a unique timestamp, and a general dataset upon initialisation which all data can be saved to
2) Run the .take_measurement() method of the Measure class:

In [ ]:
def take_measurement(self):
    time.sleep(2)

    for key, value in self.comps.items(): #Get names and component objects from dictionary
        if value.measure == True:
            #key = re.sub(r'\d', '', key) #Add this in to remove numbers from each item you pass in
            measure_func = getattr(self, key)
            measure_func()
            logging.info(f'{key} complete')

    comp.update_dataset(self.dataset)

    self.dataset.saveAllData()
    return str(self.timestamp)

For each item in the components dictionary, take_measurement checks if the item has its 'measure' attribute set to true e.g.

In [ ]:
class GenericComponent():
    def __init__(self, locator):
        device.open(locator)
        self.measure=True #Measure attribute

If measure is true, take_measurement() then looks for a function inside measure.py that is the same as the name of the item in the dictionary. For example:

In [ ]:
#Create a dictionary item in running loop
components.update({"powermeter": comp.Thor_PowerMeter(power_meter_usb_name='USB0::0x1313::0x8078::P0034379::INSTR', label='power', num_power_readings=100, bs_factor=0.5, wavelength=950)})

In [ ]:
#Inside the Measure class inside measure.py
def powermeter(self):
    self.comps['powermeter'].take_power_reading()

#This function will be called, as Thor_PowerMeter.measure = True, and take a measurement

The final part of take_measurement() saves the most recent params dictionary, any other datasets (e.g. picture) and returns the unique timestamp.


Component classes should exist inside components.py, and each component should have a measure attribute. If self.measure=True, there should be a function inside the Measure class inside Measure.py, with the name of the item you will use in the dictionary of components.

This function allows components to work in conjunction with each other, e.g.:

In [6]:
#Most recent image was saturated, so increase the filter wheel to reduce light hitting the camera. Note your filter wheel component must be called 'wheel'
cam.take_pic()
while cam.cam_saturated and cam.exposure == cam.min_exposure:
    self.comps['wheel'].increase_filter()
    time.sleep(5) #Required, wheel is slow!
    cam.take_pic()

# Advanced Users:


### Logging?
Instead of printing into the terminal (which slows down the editor), useful info is saved to a log file. You can add new info to the log file by:

In [ ]:
import logging
logging.info('newvalue:', value)

### How to create a new component?
Example 1: Connect to a new component from a different manufacturer, but the function is the same as before e.g. a new Power Meter

In [ ]:
#Add the Component inside Components.py

#Note this is an example class, not the full working class
class Grandaddy_PowerMeter(PowerMeter): #Inherit from the General Class
    def __init__(self, num_power_readings=100, bs_factor=1, wavelength=950, measure=True, label=False): #Initialisation function
       
        super().__init__(num_power_readings, bs_factor, wavelength, measure, label) #Arguments that all power meters require - passed to the parent initialisation function
           
        self.power_meter = rm.open_resource(i)
        self.power_meter.write(f'STSIZE {num_power_readings};MODE DCCONT;UNITS W;LAMBDA {wavelength}; SPREC 4096;SFREQ 500;BARGRAPH 1; AUTO 1') #Sets wawvelength, number to avg over etc.

    def take_power_reading(self): #Define the required method.
        reading = float(self.power_meter.query('STMEAN?'))/self.bs_factor
        return reading

The new component MUST be compatible with the corresponding function inside the class Measure, inside measure.py:

In [ ]:
def powermeter(self):
    self.comps['powermeter'].take_power_reading()

Hence take_power_reading() is a required method of any new type of power meter. 
Note: Sometimes there is no general class to inherit from e.g. Lasers, but refer to other working lasers to see minimum class requirements


Example 2: The new component does not directly make a measurement, but helps other components take measurements e.g. the filter wheel

In [ ]:
#Note this is an example class, not the full working class
class FilterWheel():

    def __init__(self, allowed_filter_positions=[0,5], com_port='COM6'):

        self.allowed_filter_positions = allowed_filter_positions
        self.com_port = com_port
        self.filter_wheel = ThorlabsFilterWheel(com=self.com_port)

        self.filter_wheel.initialize()
        self.filter_wheel.set_position(0)
        self.current_pos_index = 0
        self.measure = False #KEY
        params.update({'nd_filter': 0})

    def increase_filter(self):

        filter_pos = self.filter_wheel.get_position()
        self.filter_wheel.set_position(self.allowed_filter_positions[self.current_pos_index + 1])
        self.current_pos_index = self.current_pos_index + 1



* self.measure, defined at initialisation MUST be false. This ensures take_measurement() won't try call a measurement function. The component is accessible to other components inside measure.py
* Ensure when creating your components dictionary at the start of a measurement, the name of each component matches the names inside measure.py!

In [ ]:
def camera(self):
    cam = self.comps['camera']
    cam.take_pic()
    while cam.cam_saturated and cam.exposure == cam.min_exposure:
        self.comps['wheel'].increase_filter() #Filter wheel is used!
        time.sleep(5) 
        cam.take_pic()

Example 3: The new component collects data/takes a measurement

This is the same as Example 2 or 1, except you define a new function in measure.py, and __ensure self.measure=True__ at initialisation:

In [ ]:
def lasermeter(self):
    self.comps['lasermeter'].take_power_reading()

Note the name of this function should be the same as the name of the component you add to the dictionary at the start of your measurement


### Taking measurements with multiple versions of the same component

It is best to define a new function - it is unlikely there will be too many versions of the same component. e.g. If you want to measure with two powermeters:

In [ ]:
#In measure.py
def powermeter(self):
    self.comps['powermeter'].take_power_reading()

def lasermeter(self):
    self.comps['lasermeter'].take_power_reading()


#In the running loop file
components = dict()
components.update({"powermeter": comp.Thor_PowerMeter(power_meter_usb_name='USB0::0x1313::0x8078::P0034379::INSTR', label='power', num_power_readings=100, bs_factor=0.5, wavelength=950)})
components.update({"lasermeter": comp.Thor_PowerMeter(power_meter_usb_name='USB0::0x1313::0x8078::P0036521::INSTR', label='laser', num_power_readings=100, bs_factor=3/7, wavelength=784)})


Key is to make sure when data is saved, the second component does not overwrite the first component. The powermeter class lets you set a unique 'label' at startup, but for other classes, you may have to alter the function inside measure.py to assign different file names depending on the component id, or alter the experimental dataset class from pbec_analysis